In [ ]:
import os
import time
import copy
import shutil
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm
torch.backends.cudnn.benchmarks = True

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using {} device".format(device))

class dataset(Dataset):
    def __init__(self, num_classes, root, transforms):
        self.root = root
        self.labels = torch.load(os.path.join(root, 'labels'))
        self.num_classes = num_classes
        self.transforms = transforms
        
    def __len__(self):
        return self.labels.size(dim=0)
        
    def __getitem__(self, index):
        y_label = self.labels[index, :]
        image = Image.open(os.path.join(self.root, f'{index}.jpg'))
        image = self.transforms(image)
        return (image, y_label)

In [ ]:
num_classes = 28
size = 480
# Batch size for training (change depending on how much memory you have)
batch_size = 20

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False
use_amp = True

Transforms_train = transforms.Compose([
        transforms.RandomResizedCrop(size, interpolation=transforms.functional.InterpolationMode.BILINEAR),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

Transforms_val =transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

train_set = dataset(num_classes=num_classes, root=r'train', transforms=Transforms_train)
val_set = dataset(num_classes=num_classes, root=r'val', transforms=Transforms_val)

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloaders_dict = {'train': train_loader, 'val': test_loader}
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100000

    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            loop = tqdm(dataloaders[phase], leave=True)
            for inputs, labels in loop:
                # zero the parameter gradients
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    with torch.cuda.amp.autocast(enabled=use_amp):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    if phase == 'train':
                        scaler.scale(loss).backward()
                        scaler.step(optimizer)
                        scaler.update()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(outputs.argmax(1) == labels.argmax(1))
                loop.set_postfix(
                    loss=loss.item(),
                    )

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            scheduler.step(epoch_loss)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    optimizer.zero_grad()
    return model, val_acc_history

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=device)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr
        param_group['capturable'] = True

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, num_classes, feature_extract, use_pretrained=False):
        super(CustomModel, self).__init__()
        self.res = models.efficientnet_v2_s(pretrained=use_pretrained)
        set_parameter_requires_grad(self.res, feature_extract)
        num_ftrs = self.res.classifier[1].in_features
        self.res.classifier[1] = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        x = self.res(x)
        return x


model_ft = CustomModel(num_classes, feature_extract, use_pretrained=True).to(device)

In [ ]:
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)


optimizer_ft = optim.Adam(params_to_update, lr=0.0002)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, patience=5, verbose=True)

load_checkpoint("/content/drive/MyDrive/Colab Notebooks/effnet_s_BCE.pth", model_ft, optimizer_ft, lr=0.001)

In [ ]:
# Number of epochs to train for
num_epochs = 8

# Setup the loss fn
criterion = nn.BCEWithLogitsLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, scheduler=scheduler)

In [ ]:
save_checkpoint(model_ft, optimizer_ft, filename="effnet_s_BCE.pth")

=> Saving checkpoint
